## Implementation Workflow

##### ■ Step 1: Canonicalization of the MPO

- Understand the canonical forms for MPOs as presented in the paper of Parker et al.
- Design a powerpoint slide with the definitions and corresponding graphical notations.
- Explain the algorithm needed how to bring a generic MPO into the respective forms.
- Implement a function which brings an MPO into specific two-site canonical form as needed in our algorithm.

In [ ]:
"""
    site_canonicalize(mpo::Vector, center::Int)

Algorithm bringing a given MPO into the two-site canonical form with orthogonality center at sites (center, center+1).

Input:
- `mpo::Vector`: List of the local tensors of MPO to be canonicalized
- `center::Int`: Index of the orthogonality center located at site (center, center+1)

Output:
- `mpo_canonical::Vector`: List of canonicalized local tensors of the input MPO
"""
function site_canonicalize(mpo::Vector, center::Int)

end

##### ■ Step 2: Implement modified updateLeft function

- Modify the `updateLeft` function such that it accepts degree-4 tensors A and B (`updateLeftEnv`).
- The new function should also perform an additional trace over the physical legs at the very end.
- By permuting indices of the input tensors, we can reuse the same function for the update of the right environment.

In [ ]:
"""
    updateLeftEnv(V::Array{ComplexF64,3}, A::Array{ComplexF64,4}, B::Array{ComplexF64,4}, C::Array{ComplexF64,4})

Update function to compute the left environment V for given input tensors, in the following pattern,

   .-----.   
   |     |
   |  .--A-- 3
   |  |  |
   |  V--B-- 2
   |  |  |
   |  '--C-- 1
   |     |
   '-----'

Input:
- `V::Array{ComplexF64,3}` (3-leg tensor): left environment of the current site.
- `A::Array{ComplexF64,4}`, `B::Array{ComplexF64,4}`, `C::Array{ComplexF64,4}` (4-leg tensors): local MPO tensors.

Output:
- `V_new::Array{ComplexF64,3}`: Updated tensor corresponding to the left environment for the site to the right of the output legs.
"""
function updateLeftEnv(V::Array{ComplexF64,3}, A::Array{ComplexF64,4}, B::Array{ComplexF64,4}, C::Array{ComplexF64,4})

end

##### ■ Step 3: Import of the SVD function

- Import the SVD function (with unit tests) from the lecture to split the optimized two-local tensors after each step.
- Understand how contracting the decomposed tensors preserves the site-canonical form of the MPO for subsequent updates.

##### ■ Step 4: Efficient implementation of the sweeps

- The `XTRG_step` function should update $\rho(\beta)$ to $\rho(2 \beta)$ if possible and otherwise throw an error which aborts the simulation.
- After a number of `Nsweeps` iterations, the overlap $||\rho(2\beta)- \rho_{\text{opt}}(2\beta)||_F^2$ is computed to check whether the optimization has sufficiently converged as specified by the `tolerance` value. 
- Compute the inner product of the $\rho(2\beta)_{\text{diff}}$ operator while carefully performing the correct trace. The function should print a message if the result is still compatible with the `tolerance`. In this way, be can later adjust the number of `Nsweeps` that is appropriate for our simulation purposes.

In [ ]:
"""
    XTRG_step(rho::Vector, beta::Float64)

Function performing a single update of the XTRG algorithm from inverse temperatures beta ---> 2*beta

Input:
- `rho::Vector`: List of the (canonicalized) local tensors of the MPO corresponding to the quantum state at inverse temperature beta.
- `beta::Float64`: Current inverse temperature of the input state rho.
- `Nsweeps::Int`: Number of sweeps performed in the variational DMRG-type optimization along one direction of the chain.
- `tolerance:Float64`: Threshold value for which the locally optimized tensor rho_new is assumed converged.

Output:
- `rho_new::Vector`: List of (canonicalized) local tensors of the MPO corresponding to the quantum state at inverse temperature 2*beta.
- `beta::Float64`: Increased inverse temperature 2*beta for the state rho_new. 
"""
function XTRG_step(rho::Vector, beta::Float64, Nsweeps::Int, tolerance::Float64=1e-10)

    beta += beta

    try
        rho2 = square_mpo(rho)
    catch err 
        if occursin("Maximal bond dimension", err.msg)
            println("Aborting XTRG simulation", err.msg)
            return nothing

    rho2_new = deepcopy(rho2)
    # Here comes the variational DMRG-type sweeping using the above methods modifying rho_new in place
    #
    # 
    #
    # 

    # Evaluate whether the optimization has sufficiently converged
    rho2_diff = add_mpo(rho2, [-rho2_new[1], rho2_new[2:end]])

    return rho2_new, beta

end

##### ■ Step 5: Assembling the XTRG Algorithm

- This function should implement a dictionary associating the modeled inverse temperatures $\beta$ with the corresponding quantum states $\rho(\beta)$.
- It should coordinate the calls to the `XTRG_step` function and abort the simulation if the `XTRG_step` function does not return a new  state.

In [ ]:
function XTRG_algorithm(beta0::Float64, betamax::Float64)

end

From here on we can write and try out code, to be later wrapped up into functions and "official" cells of this notebook.

Here is a possible scheme for implementing `square_mpo`, the function that takes as parameters an `mpo`, an upper bound `Dmax` to the bond dimension, and a number `Nsweeps` of sweeps, and returns an mpo `mpo2` representing the square of the operator represented my `mpo`. See source/square_mpo.jl for better description of what this function `square_mpo` should do. There, the easy case where (bond dimension of `mpo`)^2 <= Dmax is already implemented (no truncation is needed, so easy!). Here let's assume (bond dimension of `mpo`)^2 > Dmax, which requires truncation and actually understanding appendix D of the paper (sigh). This scheme closely follows ideas and in particular the indexing conventions of the lecture function `dmrg_2site`! Having a tab open with the implementation of `dmrg_2site` may help understand the scheme below.

- Get as parameters `mpo` to be squared, `Dmax`, and `Nsweeps`
- Initialize `mpo2` (will be output) as a copy of `mpo` (...any better ideas?) with enlarged bond dimension = `Dmax`. one has to add a bunch of zeros (1)
- by reshaping `mpo2` to an mps (by joining its physical legs), bring it to left-canonical form. Then reshape it back to the mpo form (by disjoining physical legs) (2)
- initialize `Vlr` as `Hlr` in `dmrg_2site`. roughly speaking `Vlr[l + 1]` will contain the environment tensors $V_{L/R}$ (see fig. 17c in the paper) relative to site `l` when the algorithm/sweeping is working at a couple of site that are on the right/left of site `l`.
- Fill `Vlr[2], ..., Vlr[L + 1]` with iterative contractions of `mpo`, `mpo` and `mpo2`. For that, we can use a modified version of updateLeft, say `updateLeft_mod` (3), doing what's described in fig. 17c
- Start a for cycle with the index `itS = (1:Nsweeps)`
- RIGHT TO LEFT SWEEP: `for itL = (L:-1:2)`, update (4) as in appendix D `mpo2[itL-1]` and `mpo2[itL]`. This involves `Vlr[itL-1]` as left environment $V_L$ and `Vlr[itL + 2]` as right environment $V_R$. It also involves an SVD (5) to get `mpo2[itL-1]` and `mpo2[itL]` out of the 2-site optimized tensor ($C_i C_{i+1}$ in the paper). In particular, the updated `mpo2[itL]` must be an "MPO-right isometry", so that the updated `mpo2`  will be "MPO-site canonical" with new isometry center at site `itL - 1` (still need to really get the details of all that...). After that, update `Vlr[itL + 1]` using `updateLeft_mod` (and an index permutation since we are "updating leftwards")
- LEFT TO RIGHT SWEEP: similar to above: `for itL = (1:L-1)`, update `mpo2[itL]` and `mpo2[itL+1]` using `Vlr[itL] ` and `Vlr[itL + 3]`. Then update `Vlr[itL + 1]` using `updateLeft_mod`
- end of cycle with index itS
- `mpo2` should now approximately represent the square of the operator represented by `mpo`!! ...for big enough `Dmax` and `Nsweeps`

In [ ]:
Vlr = Array{Any}(undef,1,L+2); # L is the chain length
Vlr[1] = reshape([1],(1,1,1)); # watch out, the dummy site on the left of the first site makes indices shift by one!
Vlr[end] = reshape([1],(1,1,1));

Some "isolated" functions that we can isolate and test before implementing the whole scheme are the following:

(1) write & test a function to enlarge bond dim. of an MPO by "adding zeros"

(2) understand how to bring an mpo to left-canonical form (what does this really mean? whould we really just transform the mpo into an mps by joining physical legs, getting canonical form of mps, and get back to mpo form?)

(3) write & test `updateLeft_mod` following fig 17c. use/copy `updateLeft`

(4) write & test a function for the two-site optimization (without svd), see appendix D

(5) write & test a function to SVD a 2-site, 6-leg MPO tensor. use/copy `svd`, `svdleft` and `svdright` from lecture 